In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

# import the train test split
from sklearn.model_selection import train_test_split


price_file = "IL_Corn_Price.csv"
price_df = pd.read_csv(price_file)


# Uses the input file generated from preprocessing
input_file ='input_model_weighted_average.csv'

input_data = pd.read_csv(input_file)

# Making all column names UPPER CASE --> inline with the weather data file
input_data.columns = map(str.upper, input_data.columns)

# drop UNNAMED:0.1
input_data = input_data.drop('UNNAMED: 0', axis=1)

input_data.head()


,YEAR,MONTH,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,HUM_HIGH,HUM_AVG,...,VIS_HIGH,VIS_AVG,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM
0,1986,JAN,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,75.092035,...,18.199743,9.979591,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000
1,1986,FEB,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,99.510367,82.286915,...,16.507919,6.647287,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457
2,1986,MAR,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,99.138166,65.295138,...,18.111402,10.209948,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313
3,1986,ARP,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,99.558871,58.069374,...,19.391184,11.580387,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521
4,1986,MAY,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,95.246992,62.031159,...,19.073952,9.481744,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441


Index(['YEAR', 'MONTH', 'TMP_HIGH', 'TMP_AVG', 'TMP_LOW', 'DP_HIGH', 'DP_AVG',
       'DP_LOW', 'HUM_HIGH', 'HUM_AVG', 'HUM_LOW', 'SEALVL_HIGH', 'SEALVL_AVG',
       'SEALVL_LOW', 'VIS_HIGH', 'VIS_AVG', 'VIS_LOW', 'WIND_HIGH', 'WIND_AVG',
       'PRECIP', 'NUMDAY_RAIN', 'NUM_DAYS_SNOW', 'NUM_DAYS_FOG',
       'NUM_DAYS_THNDRSTRM'],
      dtype='object')

In [111]:
grouped_data = input_data.groupby(["YEAR"])
year_names = input_data["YEAR"].drop_duplicates().reset_index(drop=True)
year_names = year_names[year_names != 2000] # Our database has incomplete data for 2000

month_names = grouped_data.get_group(1986)["MONTH"]
month_names
weather_factors = input_data.columns[2:]
colnames = []

for mon_name in month_names:
    for factors in weather_factors:
        colnames.append(mon_name + "_" + factors)

annual_weather_input = pd.DataFrame(0, index = year_names, columns = colnames)



In [112]:


for year in year_names:
    temp_df = grouped_data.get_group(year)
    temp_list = []
    for mon_name in month_names:
        temp_list.extend(temp_df.loc[temp_df["MONTH"] == mon_name].iloc[0,2:])
    annual_weather_input.loc[year] = temp_list
    




In [114]:
temp_list

[56.208956699263105,
 28.210966908155012,
 1.8269073488677965,
 47.001875040320655,
 20.913060655064346,
 -8.8311724933041713,
 99.346199869725226,
 74.066579136591159,
 38.988302328920774,
 30.494711761976014,
 29.994242994184745,
 29.350997246749891,
 10.15259415262121,
 8.8438953578334019,
 0.059393619035925452,
 32.207612330731308,
 9.7781991637796661,
 1.2329440422206637,
 4.7998191553786205,
 8.511348052018322,
 2.4592319202202604,
 0.015259415262120647,
 66.900172312251428,
 33.807152508476591,
 6.3317052461588714,
 50.693124990895349,
 23.887684661398353,
 -6.1098053579121077,
 99.734310324375727,
 69.674338420202105,
 26.245278581878495,
 30.588809380612375,
 29.982752114132513,
 29.383167070046593,
 10.15259415262121,
 8.6546144728651146,
 0.061922114029921566,
 36.206539541291463,
 9.8244657204320127,
 0.83632979066558744,
 4.2247894481637722,
 6.8145302097922453,
 3.2992999294518683,
 0.56580018022030165,
 71.53824582798326,
 44.964953542219533,
 14.037543468262706,
 57.680

In [115]:
annual_price_input = pd.DataFrame(0, index = year_names, columns = month_names)
annual_price_discounted_input = pd.DataFrame(0, index = year_names, columns = month_names)

grouped_price = price_df.groupby(["Year"])
month_names[3] = "APR"
for year in year_names:
    temp_df = grouped_price.get_group(year)
    temp_list_1 = []
    temp_list_2 = []
    for mon_name in month_names:
        temp_list_1.append(temp_df[temp_df["Period"] == mon_name]["Value"].iloc[0])
        temp_list_2.append(temp_df[temp_df["Period"] == mon_name]["Discounted_Value"].iloc[0])
    
    annual_price_input.loc[year] = temp_list_1
    annual_price_discounted_input.loc[year] = temp_list_2

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [121]:
annual_price_input.to_csv("annual_price_input.csv")
annual_price_discounted_input.to_csv("annual_price_discounted_input.csv")
annual_weather_input.to_csv("annual_weather_input.csv")

In [122]:
annual_weather_input

,JAN_TMP_HIGH,JAN_TMP_AVG,JAN_TMP_LOW,JAN_DP_HIGH,JAN_DP_AVG,JAN_DP_LOW,JAN_HUM_HIGH,JAN_HUM_AVG,JAN_HUM_LOW,JAN_SEALVL_HIGH,...,DEC_VIS_HIGH,DEC_VIS_AVG,DEC_VIS_LOW,DEC_WIND_HIGH,DEC_WIND_AVG,DEC_PRECIP,DEC_NUMDAY_RAIN,DEC_NUM_DAYS_SNOW,DEC_NUM_DAYS_FOG,DEC_NUM_DAYS_THNDRSTRM
YEAR,,,,,,,,,,,,,,,,,,,,,
1986,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,75.092035,32.839506,30.674052,...,18.276040,8.111151,0.171012,30.035203,9.005973,13.063238,4.954492,6.141817,16.710648,0.000000
1987,51.603404,27.391903,-0.148039,38.817646,20.733436,-8.513812,98.946455,76.677755,34.704932,30.415186,...,19.391184,9.598138,0.000000,38.844028,11.646561,6.928783,9.977321,8.323399,14.877116,1.181361
1988,56.407002,25.686117,-0.995032,46.273711,17.866325,-7.806793,99.867597,73.854349,34.931524,30.699720,...,19.391184,11.411940,0.000000,29.942347,11.087768,1.484312,4.681702,6.724860,7.015422,0.394821
1989,64.166278,30.670111,5.880811,48.648697,22.362794,-1.682774,100.000000,73.753432,32.277209,30.704138,...,19.391184,10.324949,0.000000,35.840153,9.254137,13.640465,1.860475,11.521330,8.494498,0.000000
1990,67.529677,44.554956,25.075437,49.709007,25.711125,4.956941,89.797538,67.230550,27.670203,30.440353,...,18.300289,9.936429,0.123844,30.633513,10.271417,2.056510,6.864935,6.986234,12.641199,0.388022
1991,43.368253,25.461370,3.588280,36.134011,20.209888,-5.211373,99.127284,80.597620,38.417212,30.626277,...,18.300289,9.373647,0.000000,29.951122,9.403403,1.191461,5.039728,3.175232,15.066190,0.415485
1992,50.047979,30.675812,1.109954,40.819080,24.167717,-6.139658,99.814234,78.754114,38.068572,30.498681,...,18.300289,9.001303,0.061922,33.597474,9.667981,2.009294,5.907737,7.617746,11.544263,0.061922
1993,53.403603,29.051889,6.435890,48.924759,23.198574,0.352510,100.000000,79.210183,31.109257,30.658311,...,17.860550,10.103579,0.000000,28.076645,9.601381,1.076924,6.929617,7.942701,12.874551,0.000000
1994,47.582543,19.810711,-16.381349,42.600942,13.476157,-23.794517,99.814234,77.178969,41.022925,30.686980,...,17.749182,8.544477,0.109089,26.646452,8.361590,0.864822,6.241177,5.005449,18.373923,0.270226


In [123]:
annual_price_input

MONTH,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
YEAR,,,,,,,,,,,,
1986,2.35,2.38,2.35,2.34,2.45,2.45,2.04,1.77,1.52,1.39,1.47,1.54
1987,1.53,1.45,1.51,1.56,1.72,1.75,1.66,1.53,1.53,1.60,1.68,1.80
1988,1.81,1.90,1.91,1.94,2.03,2.51,2.77,2.68,2.63,2.57,2.55,2.53
1989,2.62,2.68,2.67,2.63,2.68,2.58,2.56,2.31,2.34,2.25,2.28,2.33
1990,2.31,2.34,2.44,2.61,2.71,2.74,2.71,2.55,2.37,2.26,2.22,2.29
1991,2.33,2.39,2.47,2.52,2.48,2.40,2.37,2.48,2.39,2.42,2.38,2.44
1992,2.48,2.58,2.63,2.56,2.55,2.53,2.35,2.20,2.16,2.00,2.01,2.06
1993,2.08,2.05,2.15,2.22,2.21,2.11,2.23,2.26,2.23,2.31,2.44,2.69
1994,2.71,2.86,2.82,2.75,2.67,2.71,2.31,2.17,2.18,2.03,2.00,2.17
